# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [114]:
import graphlab

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [115]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = feature.apply(lambda x: x**power)
        
    return poly_sframe
    

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [116]:
import matplotlib.pyplot as plt
%matplotlib inline

In [117]:
sales = graphlab.SFrame('kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [118]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [119]:
l2_small_penalty = 1e-5

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [120]:
poly15_data = polynomial_sframe(sales['sqft_living'],15)
poly15_features = poly15_data.column_names()
poly15_data['price'] = sales['price']
#print poly15_features
#print poly15_data.show(15)
model1 = graphlab.linear_regression.create(poly15_data, target='price', features=poly15_features ,l2_penalty =l2_small_penalty,validation_set =None )
model1.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.033999     | 2662555.737886     | 245656.462165 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,167924.858154,932257.208736
power_1,None,103.090949754,4735.64047203
power_2,None,0.134604553044,9.85916611863
power_3,None,-0.000129071365146,0.0111681953814
power_4,None,5.18928960684e-08,7.69612934514e-06
power_5,None,-7.77169308381e-12,3.40375283346e-09
power_6,None,1.71144848253e-16,9.86487827549e-13
power_7,None,4.51177961859e-20,1.85595597809e-16
power_8,None,-4.78839845626e-25,2.13680017115e-20
power_9,None,-2.33343504241e-28,1.22638027914e-24


***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [121]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [122]:
poly15_data1 = polynomial_sframe(set_1['sqft_living'],15)
poly15_features1 = poly15_data1.column_names()
poly15_data1['price'] = set_1['price']
#print poly15_features
#print poly15_data.show(15)
model11 = graphlab.linear_regression.create(poly15_data1, target='price', features=poly15_features1 ,
                                            l2_penalty =l2_small_penalty, validation_set =None )
model11.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.020913     | 2191984.900345     | 248699.117255 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,9306.4606221,1255976.06157
power_1,None,585.865823394,6003.28876462
power_2,None,-0.397305895643,11.9613259026
power_3,None,0.000141470900599,0.0131345206929
power_4,None,-1.52945989958e-08,8.84846297281e-06
power_5,None,-3.79756325772e-13,3.83736176297e-09
power_6,None,5.97481763253e-17,1.08407665704e-12
power_7,None,1.06888504767e-20,1.93500977266e-16
power_8,None,1.59344027887e-25,1.89384579435e-20
power_9,None,-6.92834984105e-29,nan


In [123]:
poly15_data2 = polynomial_sframe(set_2['sqft_living'],15)
poly15_features2 = poly15_data2.column_names()
poly15_data2['price'] = set_2['price']
#print poly15_features
#print poly15_data.show(15)
model12 = graphlab.linear_regression.create(poly15_data2, target='price', features=poly15_features2 ,
                                            l2_penalty =l2_small_penalty,
                                           validation_set =None )
model12.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.019257     | 1975178.190912     | 234533.610646 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,-25115.9044254,1573017.12178
power_1,None,783.49380028,9293.98471763
power_2,None,-0.767759302942,23.3971636819
power_3,None,0.000438766369254,0.0331210085962
power_4,None,-1.15169166858e-07,2.93724988829e-05
power_5,None,6.84281360981e-12,1.72037616446e-08
power_6,None,2.51195187082e-15,6.82484060777e-12
power_7,None,-2.06440608259e-19,1.84509003318e-15
power_8,None,-4.59673022352e-23,3.35295936015e-19
power_9,None,-2.71279236963e-29,3.78708790458e-23


In [124]:
poly15_data3 = polynomial_sframe(set_3['sqft_living'],15)
poly15_features3 = poly15_data3.column_names()
poly15_data3['price'] = set_3['price']
#print poly15_features
#print poly15_data.show(15)
model13 = graphlab.linear_regression.create(poly15_data3, target='price', features=poly15_features3 ,
                                            l2_penalty =l2_small_penalty,
                                           validation_set =None )
model13.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.017045     | 2283722.683933     | 251097.728054 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,462426.565731,nan
power_1,None,-759.251842854,nan
power_2,None,1.0286700473,nan
power_3,None,-0.000528264527386,nan
power_4,None,1.15422908385e-07,nan
power_5,None,-2.26095948062e-12,nan
power_6,None,-2.08214287571e-15,nan
power_7,None,4.08770475709e-20,nan
power_8,None,2.570791329e-23,nan
power_9,None,1.24311265196e-27,nan


In [125]:
poly15_data4 = polynomial_sframe(set_4['sqft_living'],15)
poly15_features4 = poly15_data4.column_names()
poly15_data4['price'] = set_4['price']
#print poly15_features
#print poly15_data.show(15)
model14 = graphlab.linear_regression.create(poly15_data4, target='price', features=poly15_features4 ,
                                            l2_penalty =l2_small_penalty,
                                           validation_set =None )
model14.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.020143     | 2378292.372298     | 244341.293204 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,-170240.032842,1543796.62885
power_1,None,1247.59034541,9978.42791287
power_2,None,-1.22460912177,26.7138477947
power_3,None,0.000555254626344,0.0392072641565
power_4,None,-6.3826237386e-08,3.51264407588e-05
power_5,None,-2.20215991142e-11,2.00572757169e-08
power_6,None,4.81834694285e-15,7.23449143456e-12
power_7,None,4.21461612787e-19,1.43057715883e-15
power_8,None,-7.99880736276e-23,nan
power_9,None,-1.32365897487e-26,nan


The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [126]:
poly15_data1 = polynomial_sframe(set_1['sqft_living'],15)
poly15_features1 = poly15_data1.column_names()
poly15_data1['price'] = set_1['price']
#print poly15_features
#print poly15_data.show(15)
model5 = graphlab.linear_regression.create(poly15_data1, target='price', features=poly15_features1 ,l2_penalty =1e5,
                                           validation_set =None )
model5.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.034070     | 5978778.434729     | 374261.720860 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,530317.024516,1890090.19151
power_1,None,2.58738875673,9034.21455077
power_2,None,0.00127414400592,18.000330944
power_3,None,1.74934226932e-07,0.019765845458
power_4,None,1.06022119097e-11,1.33158533722e-05
power_5,None,5.42247604482e-16,5.77475960841e-09
power_6,None,2.89563828343e-20,1.63140263499e-12
power_7,None,1.65000666351e-24,2.91195278612e-16
power_8,None,9.86081528409e-29,2.85000603886e-20
power_9,None,6.06589348254e-33,nan


In [127]:
poly15_data2 = polynomial_sframe(set_2['sqft_living'],15)
poly15_features2 = poly15_data2.column_names()
poly15_data2['price'] = set_2['price']
#print poly15_features
#print poly15_data.show(15)
model6 = graphlab.linear_regression.create(poly15_data2, target='price', features=poly15_features2 ,l2_penalty =1e5,
                                           validation_set =None )
model6.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.012667     | 2984894.541944     | 323238.809634 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,519216.897383,2167962.96521
power_1,None,2.04470474182,12809.1515268
power_2,None,0.0011314362684,32.2464286313
power_3,None,2.93074277549e-07,0.0456480218892
power_4,None,4.43540598453e-11,4.0481752482e-05
power_5,None,4.80849112204e-15,2.37105608016e-08
power_6,None,4.53091707826e-19,9.40612881847e-12
power_7,None,4.16042910575e-23,2.54293917341e-15
power_8,None,3.90094635128e-27,4.62111417353e-19
power_9,None,3.7773187602e-31,5.21943862496e-23


In [128]:
poly15_data3 = polynomial_sframe(set_3['sqft_living'],15)
poly15_features3 = poly15_data3.column_names()
poly15_data3['price'] = set_3['price']
#print poly15_features
#print poly15_data.show(15)
model7 = graphlab.linear_regression.create(poly15_data3, target='price', features=poly15_features3 ,l2_penalty =1e5,
                                           validation_set =None )
model7.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.016606     | 3695342.767093     | 350033.521294 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,522911.518048,nan
power_1,None,2.26890421877,nan
power_2,None,0.00125905041842,nan
power_3,None,2.77552918155e-07,nan
power_4,None,3.2093309779e-11,nan
power_5,None,2.87573572364e-15,nan
power_6,None,2.50076112671e-19,nan
power_7,None,2.24685265906e-23,nan
power_8,None,2.09349983135e-27,nan
power_9,None,2.00435383296e-31,nan


In [129]:
poly15_data4 = polynomial_sframe(set_4['sqft_living'],15)
poly15_features4 = poly15_data4.column_names()
poly15_data4['price'] = set_4['price']
#print poly15_features
#print poly15_data.show(15)
model8 = graphlab.linear_regression.create(poly15_data4, target='price', features=poly15_features4 ,l2_penalty =1e5,
                                           validation_set =None )
model8.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.022863     | 3601895.280124     | 323111.582889 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,513667.087087,2041482.90233
power_1,None,1.91040938244,13195.2548642
power_2,None,0.00110058029175,35.3258081465
power_3,None,3.12753987879e-07,0.051846828738
power_4,None,5.50067886825e-11,4.64504371162e-05
power_5,None,7.20467557825e-15,2.65233027965e-08
power_6,None,8.24977249384e-19,9.56673326956e-12
power_7,None,9.06503223498e-23,1.89176395105e-15
power_8,None,9.95683160453e-27,nan
power_9,None,1.10838127982e-30,nan


These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [130]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)
print train_valid_shuffled

+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 2780400035 | 2014-05-05 00:00:00+00:00 | 665000.0 |   4.0    |    2.5    |
| 1703050500 | 2015-03-21 00:00:00+00:00 | 645000.0 |   3.0    |    2.5    |
| 5700002325 | 2014-06-05 00:00:00+00:00 | 640000.0 |   3.0    |    1.75   |
| 0475000510 | 2014-11-18 00:00:00+00:00 | 594000.0 |   3.0    |    1.0    |
| 0844001052 | 2015-01-28 00:00:00+00:00 | 365000.0 |   4.0    |    2.5    |
| 2781280290 | 2015-04-27 00:00:00+00:00 | 305000.0 |   3.0    |    2.5    |
| 2214800630 | 2014-11-05 00:00:00+00:00 | 239950.0 |   3.0    |    2.25   |
| 2114700540 | 2014-10-21 00:00:00+00:00 | 366000.0 |   3.0    |    2.5    |
| 2596400050 | 2014-07-30 00:00:00+00:00 | 375000.0 |   3.0    |    1.0    |
| 4140900050 | 2015-01-26 00:00:00+00:00 | 440000.0 |   4.0    |    1.75   |

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [131]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end), end-start

0 (0, 1938) 1938
1 (1939, 3878) 1939
2 (3879, 5817) 1938
3 (5818, 7757) 1939
4 (7758, 9697) 1939
5 (9698, 11636) 1938
6 (11637, 13576) 1939
7 (13577, 15515) 1938
8 (15516, 17455) 1939
9 (17456, 19395) 1939


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

## train_valid_shuffled[0:10] # rows 0 to 9

Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [132]:
validation4 = train_valid_shuffled[5818:7757]
#print validation4

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [133]:
print int(round(validation4['price'].mean(), 0))

536353


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [134]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
#print first_two.append(last_two)
#print train_valid_shuffled.column_names()

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [135]:
n=len(train_valid_shuffled)
firstv4 = train_valid_shuffled[0:5817]
lastv4 = train_valid_shuffled[7758:19395]
train4 = firstv4.append(lastv4)

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [136]:
print int(round(train4['price'].mean(), 0))

539469


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [137]:
def k_fold_cross_validation(k, l2_penalty, data, features_list):
    
    n = len(data)

    #print n

    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        #print i, (start,end)
        val_set = data[start:end]
        train_set = data[end+1:n].append(data[0:start])
       
        
        tm1 = graphlab.linear_regression.create(train_set, target = 'price', features =features_list,
                                                      l2_penalty = l2_penalty, validation_set =None, verbose=False)
        
        vpredictions = tm1.predict(val_set)
        val_error = vpredictions - val_set['price']
        RSS_V = sum(val_error * val_error)
        
        
        Sum_RSS_V = 0.0
        Sum_RSS_V = Sum_RSS_V+RSS_V
        
        Avg_RSS_V = Sum_RSS_V/(i+1)
        
        print "RSS_V"+str(i)+":"+ str(RSS_V)+","+"Avg_RSS_V"+str(i)+":"+str(Avg_RSS_V)
    
    
    

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [138]:
import numpy as np
poly_data_15 = polynomial_sframe(train_valid_shuffled['sqft_living'],15)
poly_features = poly_data_15.column_names()
poly_data_15['price'] = train_valid_shuffled['price']

#print poly_data_15

l2_penalty = np.logspace(1,7,num=13)
 
for l2_p in l2_penalty:    
    print "l2_penalty:"+ str(l2_p)
    k_fold_cross_validation(10, l2_p, poly_data_15, poly_features)



l2_penalty:10.0
RSS_V0:1.09810751769e+14,Avg_RSS_V0:1.09810751769e+14
RSS_V1:1.10750236808e+14,Avg_RSS_V1:5.53751184039e+13
RSS_V2:9.56783411879e+13,Avg_RSS_V2:3.1892780396e+13
RSS_V3:1.22811504881e+14,Avg_RSS_V3:3.07028762202e+13
RSS_V4:1.11515144154e+14,Avg_RSS_V4:2.23030288309e+13
RSS_V5:3.83506121376e+15,Avg_RSS_V5:6.39176868959e+14
RSS_V6:1.35375897177e+14,Avg_RSS_V6:1.93394138824e+13
RSS_V7:1.29067020652e+14,Avg_RSS_V7:1.61333775814e+13
RSS_V8:1.1433362824e+14,Avg_RSS_V8:1.27037364711e+13
RSS_V9:1.52915174505e+14,Avg_RSS_V9:1.52915174505e+13
l2_penalty:31.6227766017
RSS_V0:1.09833680833e+14,Avg_RSS_V0:1.09833680833e+14
RSS_V1:1.10820988244e+14,Avg_RSS_V1:5.54104941218e+13
RSS_V2:9.56837474844e+13,Avg_RSS_V2:3.18945824948e+13
RSS_V3:1.22731190256e+14,Avg_RSS_V3:3.0682797564e+13
RSS_V4:1.11665299668e+14,Avg_RSS_V4:2.23330599337e+13
RSS_V5:1.8083642767e+15,Avg_RSS_V5:3.01394046116e+14
RSS_V6:1.35358232975e+14,Avg_RSS_V6:1.9336890425e+13
RSS_V7:1.29404343536e+14,Avg_RSS_V7:1.61755429

***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [139]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`.  This way, your final model will be trained on the entire dataset.

In [143]:
poly_t_data_15 =polynomial_sframe(train_valid['sqft_living'],15)

ply_features = poly_t_data_15.column_names()
poly_t_data_15['price'] = train_valid['price']



tm2 = graphlab.linear_regression.create(poly_t_data_15, target = 'price', features = ply_features, l2_penalty = 100,
                                       validation_set =None)

tm6 = tm2.predict(test)
test_error = tm6 - test['price']

RSS = sum(test_error*test_error)
print RSS





Linear regression:

--------------------------------------------------------

Number of examples          : 19396

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.025358     | 2347542.318320     | 247782.989308 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

2.52897427447e+14


***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***